# ResNetV2
### Dataset : Reconstructed Augmented Dataset.
### Loss Function : Label Smothing 0.2로 설정.
### KFold : 5, Stratified
### checkpoint save path : ./resnet_augmendted_labelsmoothing_0918
### date : 09/18 06:11


In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from filemanager import get_file_paths as gfp
import matplotlib.font_manager as fm
      
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256
IMAGE_CHANNEL = 3
NUM_IMAGE = 10000

NUM_CLASS = 50

# 레이블 값들 가져오기 (데이터 시각화용)

In [ ]:
dataset = gfp.File_Control.searchAllFilesInDirectoryByDir("/raid/food_competition_KT_set1/train", "jpg")
dataset.sort()
label_dict = {}
for i in range(len(dataset)):
    label = os.path.dirname(dataset[i][0])
    label = os.path.basename(label)
    label_dict[str(i)] = label
print(label_dict)

# npy 파일 불러오기

In [ ]:
x = np.load('/raid/food_competition_KT_set1/kfood_img.npy')
y = np.load('/raid/food_competition_KT_set1/kfood_y.npy')

x_augmented = np.load("/raid/food_competition_KT_set1/kfood_orgimg_augmented_0918.npy")
y_augmented = np.load("/raid/food_competition_KT_set1/kfood_orgy_augmented_0918.npy")


# 시각화 코드

In [ ]:
path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
prop = fm.FontProperties(fname=path, size=18)

def show_batch(image_batch, label_batch, num_photo=5):
    plt.figure(figsize=(25, 25))
    for n in range(num_photo*num_photo):
        ax = plt.subplot(num_photo, num_photo, n + 1)
        ax.set_title(label_dict[str(np.argmax(label_batch[n]))], fontproperties=prop)
        plt.imshow(image_batch[n] / 255.0)
        plt.axis("off")
#print(int(NUM_IMAGE/BATCH_SIZE))

# 데이터 시각화

In [ ]:
batch_x = x[:25]
batch_y = y[:25]
show_batch(batch_x, batch_y)

batch_x = x_augmented[:25]
batch_y = y_augmented[:25]
show_batch(batch_x, batch_y)

# Import Packages (TF)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler

# 데이터 표준화 (0~1)

In [ ]:
x = x / 255
print(x.shape, y.shape)
print(np.min(x), np.max(x))

x_augmented = x_augmented / 255
print(x_augmented.shape, y_augmented.shape)
print(np.min(x_augmented), np.max(x_augmented))

# 기본 모델 구성

In [ ]:
def resnet_model(img_height, img_width, num_classes):
    model = ResNet50V2(
        include_top=True,
        input_shape=(img_width, img_height, 3),
        weights=None,
        classes=50,
    )
    
    return model

model = resnet_model(IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CLASS)
model.summary()

# 컴파일 및 하이퍼 파라미터 설정

# K-Fold Cross Validation Code 작성

In [ ]:
#dataset = read_dataset(100, './dataset', 'kt_food')

NUM_FOLD = 5
EPOCHS = 30

kfold = StratifiedKFold(n_splits=NUM_FOLD, shuffle=True, random_state=30)

fold_no = 1
results = []

CHECKPOINT_PATH = './resnet_augmented2_labelsmoothing_0918'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=CHECKPOINT_PATH,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

model_early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

for train_index, test_index in kfold.split(x, y.argmax(1)):
    
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    x_train = np.concatenate((x_augmented[train_index], x_train))
    y_train = np.concatenate((y_augmented[train_index], y_train))
    
    
    print("test index : ", test_index)
    print("test index shape : ", test_index.shape)
    print("shapes : ", x_train.shape, y_train.shape, x_test.shape, y_test.shape)
    #show_batch(x_test[:25] * 255, y_test[:25])
    
    np.random.seed(0)
    np.random.shuffle(x_train)
    np.random.seed(0)
    np.random.shuffle(y_train)
    
    model = resnet_model(IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CLASS)
    loss_f = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.2)
    
    model.compile(optimizer='adam',
              loss=loss_f,
              metrics=['accuracy'])
    
    print('-' * 50)
    print(f'Training for fold {fold_no} ...')
    print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
    
    hist = model.fit(x=x_train, 
              y=y_train, 
              validation_data=(x_test, y_test), 
              epochs=EPOCHS, 
              batch_size=32,
              callbacks=[model_checkpoint_callback, model_early_stopping_callback]
            )
    
    eval_results = model.evaluate(x_test, y_test)
    print("test loss, test acc : ", eval_results)
    results.append(hist)
    #del model
    fold_no = fold_no + 1
    break

In [ ]:
import pickle

with open('ResNetV2-Reconstructed-augmented-2/hist_fold2.pickle', 'wb') as f:
    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)